In [ ]:
import numpy as np

NF4_CENTROIDS = np.array([
    -1.0000, -0.6962, -0.5257, -0.3948,
    -0.2847, -0.1848, -0.0918, -0.0003,
     0.0003,  0.0918,  0.1848,  0.2847,
     0.3948,  0.5257,  0.6962,  1.0000
])

def nf4_quantize(z):
    index = np.argmin(np.abs(z - NF4_CENTROIDS))
    return index, NF4_CENTROIDS[index]

z = 0.25
idx, quantized_value = nf4_quantize(z)
print(f"Giá trị gốc: {z}, mapping sang: {quantized_value}, index: {idx}")


Giá trị gốc: 0.25, mapping sang: 0.2847, index: 11


In [8]:
from datasets import load_dataset

In [12]:
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [13]:
# Shuffle the dataset and slice it
dataset = dataset.shuffle(seed=42).select(range(1500))

In [ ]:
chunks = dataset[2]["text"].split("###")

In [45]:
dataset[2]["text"].split("###")

['',
 ' Human: Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?',
 ' Assistant: Thanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like knife skills, cooking techniques, menu development, etc. Mention the types of cuisine you specialize in.\n\n3) Have you worked in a professional kitchen before? If so, mention 

In [28]:
chunks

['',
 ' Human: Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?',
 ' Assistant: Thanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like knife skills, cooking techniques, menu development, etc. Mention the types of cuisine you specialize in.\n\n3) Have you worked in a professional kitchen before? If so, mention 

In [ ]:
# # Shuffle the dataset and slice it
# dataset = dataset.shuffle(seed=42).select(range(1500))

# Define a function to transform the data
def transform_conversation(example):
        conversation_text = example['text']
        segments = conversation_text.split('###')

        reformatted_segments = []

        # Iterate over pairs of segments
        for i in range(1, len(segments) - 1, 2):
            human_text = segments[i].strip().replace('Human:', '').strip()

            # Check if there is a corresponding assistant segment before processing
            if i + 1 < len(segments):
                assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

                # Apply the new template
                reformatted_segments.append(f'"role" {human_text} [/INST] {assistant_text} </s>')
            else:
                # Handle the case where there is no corresponding assistant segment
                reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

        return {'conversations': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

Map: 100%|██████████| 1500/1500 [00:00<00:00, 10843.19 examples/s]


In [58]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1500
})

In [52]:
transformed_dataset[2]["text"]

'<s>[INST] Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me? [/INST] Thanks for the information. Unfortunately, your initial message does not contain enough details or structure to compose a compelling professional email to a potential employer. Here are some suggestions for how we can improve the message:\n\n1) Let\'s first clarify the type of cooking position you are interested in. Are you looking for a job as a line cook, sous chef, executive chef, or something else? This will help determine how to emphasize relevant experience and skills.\n\n2) What specific experience do you have? For example, "Over 10 years of experience preparing authentic Sichuan and Cantonese cuisine." Highlight any relevant skills, like knife skills, cooking techniques, menu development, etc. Mention the types of cuisine you specialize in.\n\n3) Have you worked in a professional kitchen before? If so, mention the name and

In [ ]:
#use to reformat dataset to suitable form with llama 3.2
def new_transform_conversations(example):
    text_conversation = example["text"]
    segments = text_conversation.split("###")
    reformatted_segments = []
    for i in range(1,len(segments)-1,2):       
        human_text = segments[i].replace('Human:', '').strip()
        if(i+1 < len(segments)):
            assistant_text = segments[i+1].replace('Assistant:', '').strip()
        # Apply the new template
            reformatted_segments.append(f'<|start_header_id|>user<|end_header_id|> {human_text} <|eot_id|> <|start_header_id|>assistant<|end_header_id|> {assistant_text} <|eot_id|>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<|start_header_id|>user<|end_header_id|> {human_text} <|eot_id|>')
    return "".join(reformatted_segments)

In [67]:
# Apply the transformation
new_transformed_dataset = dataset.map(new_transform_conversations, remove_columns="text")

Map: 100%|██████████| 1500/1500 [00:00<00:00, 12693.37 examples/s]


In [71]:
new_transformed_dataset[2]["text"].endswith(".")

False